In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("logistic-regression").getOrCreate()

22/01/14 23:42:13 WARN Utils: Your hostname, wshid-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.18 instead (on interface en0)
22/01/14 23:42:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/14 23:42:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/14 23:42:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# pipeline을 만들기 위해 라이브러리 로드
# Tokenize가 pipeline을 적용하기 좋은 예시
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

In [6]:
# spark docs 예시와 동일
# id, text column, label(prediction)
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0), # spark가 나올때 1.0을 예측
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

In [7]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
# tokenizer.getOutputCol의 값을 그대로 활용하기 위해 'words'를 사용하지 않음
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

In [8]:
lr = LogisticRegression(maxIter=30, regParam=0.001)

In [9]:
# pipeline 생성, tokenizer, hashingTF, lr stage를 나열
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [10]:
# pipeline의 결과로 모델 출력
model = pipeline.fit(training)

22/01/14 23:46:01 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/01/14 23:46:01 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/01/14 23:46:01 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/01/14 23:46:01 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [12]:
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

In [13]:
prediction = model.transform(test)

In [14]:
prediction.select(["id", "text", "probability", "prediction"]).show()

+---+------------------+--------------------+----------+
| id|              text|         probability|prediction|
+---+------------------+--------------------+----------+
|  4|       spark i j k|[0.63102699631690...|       0.0|
|  5|             l m n|[0.98489377609773...|       0.0|
|  6|spark hadoop spark|[0.13563147748816...|       1.0|
|  7|     apache hadoop|[0.99563405823116...|       0.0|
+---+------------------+--------------------+----------+

